## Using APIs from other frontier models

Help with preventing us of spending money on OpenAI API

In [86]:
!pip install -q youtube-transcript-api

In [87]:
import os
from dotenv import load_dotenv
from IPython.display import Markdown, display
from openai import OpenAI
from urllib.parse import urlparse, parse_qs
from youtube_transcript_api import YouTubeTranscriptApi

In [88]:
load_dotenv(override=True)
api_key = os.getenv("GOOGLE_API_KEY")
gemini_base = "https://generativelanguage.googleapis.com/v1beta/openai/"

In [89]:
gemini = OpenAI(api_key=api_key, base_url=gemini_base)

In [136]:
system_prompt = """
You are a transcript-based YouTube summarizer.

Your job:
- Summarize ONLY from the transcript text provided by the user.
- Work for any topic (news, music, podcasts, tutorials, interviews, politics, etc.).

Rules:
1) Do not invent facts, names, dates, or speaker identities.
2) If something is unclear or likely mistranscribed, label it as [unclear].
3) If the transcript is noisy/incomplete, say what is uncertain.
4) Keep the summary concise and neutral.
5) Preserve important proper nouns exactly as they appear in transcript.

Output format:
- (1-2 sentences)
- Key points (4-8 bullets)
- People/organizations mentioned
- Claims that could not be verified from transcript
"""

#### Make a function to transcript a youtube video

In [132]:
def extract_video_id(url: str) -> str:
    p = urlparse(url)
    if p.netloc in ("youtu.be", "www.youtu.be"):
        return p.path.lstrip("/")
    if "youtube.com" in p.netloc:
        return parse_qs(p.query).get("v", [None])[0]
    return None

In [173]:
from youtube_transcript_api._errors import TranscriptsDisabled, NoTranscriptFound

def summarize_youtube_video(ai, video_url, model_=None):
    video_id = extract_video_id(video_url)
    if not video_id:
        raise ValueError("Invalid YouTube URL")
    ytt = YouTubeTranscriptApi()
    try:
        transcript = ytt.fetch(video_id, languages=['en'])
    except (TranscriptsDisabled, NoTranscriptFound):
        raise ValueError("Transcript not available for this video")
    transcript_text = " ".join(x.text for x in transcript)
    if ai is gemini:
        m = "gemini-2.5-flash"
    elif ai is ollama:
        if model_ == "deepseek":
            m = "deepseek-r1:1.5b"
        elif model_ == "llama3.2":
            m = "llama3.2"
        elif not model_:
            raise ValueError("For Ollama, please specify a model (e.g. 'deepseek')")
    else:
        raise ValueError("Unsupported model")
    response = ai.chat.completions.create(
        model = m,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": f"Summarize this transcript:\n\n {transcript_text}"}
        ]
    )
    print(f"Model used: {m}")
    return response.choices[0].message.content

In [157]:
def summarize_video(ai, video_url, model_=None):
    summary = summarize_youtube_video(ai, video_url, model_)
    display(Markdown(f"### Summary"))
    display(Markdown(summary))


In [159]:
video_url = "https://www.youtube.com/watch?v=c4pQVTVlaKc"

summarize_video(gemini, video_url)

Model used: gemini-2.5-flash


### Summary

The speaker details a series of aggressive actions taken against drug cartels and the government of Venezuela, including designating cartels as foreign terrorist organizations and fentanyl as a weapon of mass destruction. A new military campaign is credited with stopping record amounts of drugs, eliminating a major cartel kingpin, and culminating in the defeat and capture of Venezuelan dictator Nicholas Maduro. The speaker also notes cooperation with Deli Rodriguez, described as the new president of Venezuela, to foster economic gains and hope for the country.

Key points:
*   Large parts of Mexico are controlled by murderous drug cartels, which have been designated as foreign terrorist organizations.
*   Illicit fentanyl has been declared a weapon of mass destruction.
*   A new military campaign has reportedly stopped record amounts of drugs, with sea routes virtually completely blocked.
*   One of the most sinister cartel kingpins has been taken down.
*   America's armed forces defeated an [unclear] enemy, ending the reign of Nicholas Maduro, who was brought to face American justice.
*   This is described as a colossal victory for US security and a new beginning for Venezuela.
*   The US is working with the "new president of Venezuela," Deli Rodriguez, for economic gains.
*   Maduro's heavily protected military fortress, guarded by thousands of soldiers and Russian and Chinese military technology, was swiftly descended upon.

People/organizations mentioned:
*   Nicholas Maduro
*   Deli Rodriguez (new president of Venezuela)
*   Drug cartels
*   America's armed forces
*   Russian military technology
*   Chinese military technology

Claims that could not be verified from transcript:
*   "stopped record amounts of drugs coming into our country and virtually stopped it completely coming in by water or sea."
*   "America's armed forces overwhelmed all defenses and utterly defeated a enemy. good fighters"
*   "This was an absolutely colossal victory for the security of the United States"
*   "This was a major military installation protected by thousands of soldiers and guarded by Russian and Chinese military technology."

### Let's try Ollama!

(Ollama 3.3 or 4 are too big for my computer.)
Let's use llama3.2, smaller but effective

In [46]:
!ollama pull llama3.2

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 2.3 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   0% ▕                  ▏ 7.5 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  11 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  14 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  20 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  21 MB/2.0 GB                  pulling manifest 
pulling dde5aa3fc5ff:   1% ▕                  ▏  26 MB/2.0 GB                

In [47]:
!ollama --version

ollama version is 0.17.0


In [48]:
!ollama list

NAME               ID              SIZE      MODIFIED       
llama3.2:latest    a80c4f17acd5    2.0 GB    57 seconds ago    
phi3:latest        4f2222927938    2.2 GB    4 days ago        
gemma3:270m        e7d36fb2c3b3    291 MB    4 days ago        


In [51]:
#To check if Ollama is effectivelly running:
import requests
requests.get("http://localhost:11434").content

b'Ollama is running'

In [52]:
ollama = OpenAI(base_url="http://localhost:11434/v1", api_key="ollama")

In [163]:
video_url = "https://www.youtube.com/watch?v=c4pQVTVlaKc"

summarize_video(ollama, video_url, "llama3.2")

Model used: llama3.2


### Summary

Here is a summary of the transcript:

The president stated that their new military campaign has stopped record amounts of drugs from entering the country, almost completely halted smuggling by water or sea, and successfully taken down a powerful cartel kingpin, Nicholas Maduro, who was overthrown and brought to face American justice. The success of this operation is seen as a colossal victory for US security and opens up a new beginning for Venezuela.

Key points:
• Large parts of Mexico have been controlled by murderous drug cartels.
• The president declared illicit fentinol a weapon of mass destruction.
• A military campaign has stopped record amounts of drugs from entering the country.
• A powerful cartel kingpin, Nicholas Maduro, was taken down and overthrown.
• Maduro's regime was described as having a "heavily protected" military fortress.

People/organizations mentioned:
- President (implied)
- Deli Rodriguez, new president of Venezuela
- Mexican territory
- Russia
- China

Claims that could not be verified from transcript:
- [unclear] definition or existence of the chemical compound "fentinol"

### Let's try DeepSeek!

In [170]:
!ollama rm deepseek-r1:7b

deleted 'deepseek-r1:7b'


⠙ 


In [172]:
!ollama pull deepseek-r1:1.5b

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠸ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling aabd4debf0c8:   0% ▕                  ▏  16 KB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏ 6.5 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   1% ▕                  ▏  14 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  18 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   2% ▕                  ▏  25 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  32 MB/1.1 GB                  pulling manifest 
pulling aabd4debf0c8:   3% ▕                  ▏  38 MB/1.1

In [179]:
summarize_video(ollama, video_url, "deepseek")

Model used: deepseek-r1:1.5b


### Summary

(1-2 sentences)  
The video discusses how large territories controlled by drug cartels used fentinol as a weapon to stop illegal drug flow and take down key figures, ultimately defeating Donald Maduro in the US. Key people mentioned include drug cartel members and the use of fentinol.  

People/organizations involved: Drug cartels, military forces. Claims not verifiable due to unknown individuals and context.  
Unclear aspects: [large state terms translated as "territories," other variables possibly implied.]